<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-08 13:02:12
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.37 C
-------------------
Today PnL: -25.25 K (-0.18%)
Current PnL: -13.46 L (-9.38%)
CY Booked + Current PnL: -6.50 L (-4.53%)
-------------------
Total profit:  4.67 L
Total loss:  -18.13 L
-------------------
Total Booked + Current PnL: 20.48 L (17.57%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.09%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 71.97 L (52.66%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 10.73%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,APOLLOHOSP,6904.43,8285.00,-16.41,H-LC,91.67,113888.0,10322.0,10387.0,-0.31,9.97,9.12,20.00,32.0,0.99,0.83,25.44,X40N,BTT,HEALTHCARE
39,HINDALCO,651.95,761.55,-19.90,H-LC,85.42,110208.0,5896.0,11638.0,-0.48,5.65,10.56,16.81,27.0,0.51,0.81,22.61,X5K,ATH,METALS
71,SBIN,760.30,863.00,-11.51,M-LC,30.21,214703.0,13224.0,13999.0,0.42,6.56,6.52,13.51,77.0,0.94,1.57,19.99,XY25,NTT,BANKS
92,VIPIND,488.80,489.00,-212.52,H-SC,98.96,81178.0,-14138.0,14174.0,0.36,-14.83,17.46,0.04,157.0,-1.00,0.59,60.12,OX40N,NTT,MISC
37,HCLTECH,1647.60,1959.14,17.72,H-LC,76.04,104566.0,4062.0,14942.0,0.22,4.04,14.29,18.91,34.0,0.27,0.77,26.19,X40,ATH,IT


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-31.29,M-MC,8.33,179654.0,-154.0,112571.0,-0.77,-0.09,62.66,62.52,173.0,-0.00,1.32,33.97,XR,NTT,BANKS
38,HEROMOTOCO,4311.81,6039.03,-6.19,H-MC,35.42,150622.0,-291.0,60746.0,-0.08,-0.19,40.33,40.06,103.0,-0.00,1.10,23.20,AR,ATH,AUTO
60,NATIONALUM,189.63,247.44,-50.15,H-MC,54.17,100737.0,233.0,30402.0,0.80,0.23,30.18,30.49,97.0,0.01,0.74,32.71,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.55,NaN,NaN,50.00,18492.0,-31101.0,24.0,3.39,-62.71,0.13,-62.66,270.0,-1295.88,0.14,24.01,XY24,ATH,MISC
40,HINDUNILVR,2413.81,2723.00,-13.79,X-LC,61.46,269965.0,-382.0,35014.0,3.04,-0.14,12.97,12.81,5.0,-0.01,1.98,12.90,XY25,NTT,FMCG
85,TRIDENT,37.35,48.00,25.90,M-SC,73.96,79089.0,-13166.0,39473.0,2.89,-14.27,49.91,28.51,233.0,-0.33,0.58,35.33,XR,NTT,TEXTILES
13,BATAINDIA,1550.24,2096.00,-30.44,M-SC,68.75,103982.0,-24688.0,69990.0,1.99,-19.19,67.31,35.20,205.0,-0.35,0.76,6.09,X40,NTT,FOOTWEAR
32,ENRIN,1377.95,3059.90,NaN,NaN,95.83,76498.0,42049.0,0.0,1.99,122.06,0.00,122.06,274.0,inf,0.56,16.98,AR,ATH,ELECTRICAL


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-57.75,L-SC,44.79,63360.0,-50189.0,90237.0,-3.17,-44.20,142.42,35.27,265.0,-0.56,0.46,55.08,XR,NTT,HOTELS
65,RELAXO,902.64,1176.00,-30.28,H-SC,92.71,63034.0,-52504.0,87497.0,-2.74,-45.44,138.81,30.28,131.0,-0.60,0.46,23.14,X40N,NTT,FOOTWEAR
94,WHIRLPOOL,1167.49,2270.00,-40.20,M-SC,37.50,197127.0,25506.0,136570.0,-2.32,14.86,69.28,94.43,201.0,0.19,1.44,45.49,XR,NTT,DURABLES
19,CAMPUS,294.86,393.00,-27.11,M-SC,4.17,151895.0,-15880.0,71725.0,-2.16,-9.47,47.22,33.28,206.0,-0.22,1.11,17.60,XY24,NTT,FOOTWEAR
3,ALKYLAMINE,2347.98,4546.37,8.61,X-SC,97.92,101063.0,100.0,94433.0,-1.99,0.10,93.44,93.63,67.0,0.00,0.74,50.53,SR,ATH,CHEMICALS


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,47.02,M-SC,25.00,127278.0,-15201.0,15821.0,0.30,-10.67,12.43,0.44,247.0,-0.96,0.93,50.22,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-29.37,M-SC,27.08,150689.0,-20452.0,20403.0,-0.10,-11.95,13.54,-0.03,191.0,-1.00,1.10,8.30,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,93.75,148912.0,-25647.0,25687.0,0.01,-14.69,17.25,0.02,133.0,-1.00,1.09,29.13,OX40N,NTT,PAINTS
92,VIPIND,488.80,489.0,-212.52,H-SC,98.96,81178.0,-14138.0,14174.0,0.36,-14.83,17.46,0.04,157.0,-1.00,0.59,60.12,OX40N,NTT,MISC
81,TATAELXSI,7332.28,7332.0,-17.82,X-MC,10.42,86282.0,-16370.0,16368.0,-0.19,-15.95,18.97,-0.00,59.0,-1.00,0.63,31.16,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-18.38,X-LC,48.96,266113.0,-29821.0,123743.0,-0.24,-10.08,46.50,31.73,1.0,-0.24,1.95,6.51,X40,BTT,IT
48,INFY,1461.46,2275.00,-8.11,X-LC,80.21,294366.0,32765.0,112860.0,1.08,12.52,38.34,55.67,2.0,0.29,2.16,19.39,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-13.79,X-LC,61.46,269965.0,-382.0,35014.0,3.04,-0.14,12.97,12.81,5.0,-0.01,1.98,12.90,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,17.58,X-LC,65.62,269537.0,40304.0,26981.0,-0.42,17.58,10.01,29.35,10.0,1.49,1.97,28.07,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-2.21,X-LC,28.12,82575.0,-27597.0,116670.0,0.76,-25.05,141.29,80.85,11.0,-0.24,0.60,21.62,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3059.90,NaN,NaN,95.83,76498.0,42049.0,0.0,1.99,122.06,0.00,122.06,274.0,inf,0.56,16.98,AR,ATH,ELECTRICAL
28,DIGITIDE,188.38,237.72,NaN,NaN,29.17,49855.0,8600.0,2204.0,-0.37,20.85,4.42,26.19,272.0,3.90,0.37,17.38,XY24,ATH,IT
88,UNIONBANK,123.87,163.00,-2.93,M-LC,21.88,170175.0,29335.0,15163.0,-1.64,20.83,8.91,31.59,88.0,1.93,1.25,47.82,XY24,NTT,BANKS


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,93.57,M-MC,55.21,231919.0,6957.0,160697.0,0.91,3.09,69.29,74.53,189.0,0.04,1.70,34.65,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.64,24.30,H-SC,32.29,127159.0,1862.0,57209.0,-1.63,1.49,44.99,47.15,143.0,0.03,0.93,59.08,MH,ATH,POWER
20,CAMS,3643.00,5250.99,2.98,H-SC,23.96,115668.0,13664.0,31358.0,-0.27,13.40,27.11,44.14,126.0,0.44,0.85,31.81,X40N,ATH,MISC
95,WIPRO,243.46,420.00,-2.41,M-LC,89.58,167152.0,16207.0,93254.0,0.69,10.74,55.79,72.51,71.0,0.17,1.22,15.14,XR,NTT,IT
88,UNIONBANK,123.87,163.00,-2.93,M-LC,21.88,170175.0,29335.0,15163.0,-1.64,20.83,8.91,31.59,88.0,1.93,1.25,47.82,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3059.90,NaN,NaN,95.83,76498.0,42049.0,0.0,1.99,122.06,0.00,122.06,274.0,inf,0.56,16.98,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-40.20,M-SC,37.50,197127.0,25506.0,136570.0,-2.32,14.86,69.28,94.43,201.0,0.19,1.44,45.49,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-2.41,M-LC,89.58,167152.0,16207.0,93254.0,0.69,10.74,55.79,72.51,71.0,0.17,1.22,15.14,XR,NTT,IT
45,INDIAMART,2327.09,4911.36,-48.26,H-SC,69.79,134837.0,11501.0,125466.0,0.22,9.33,93.05,111.05,123.0,0.09,0.99,32.12,AR,ATH,MISC
54,KPIGREEN,497.21,731.64,24.30,H-SC,32.29,127159.0,1862.0,57209.0,-1.63,1.49,44.99,47.15,143.0,0.03,0.93,59.08,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.55,NaN,NaN,50.00,18492.0,-31101.0,24.0,3.39,-62.71,0.13,-62.66,270.0,-1295.88,0.14,24.01,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-57.75,L-SC,44.79,63360.0,-50189.0,90237.0,-3.17,-44.20,142.42,35.27,265.0,-0.56,0.46,55.08,XR,NTT,HOTELS
32,ENRIN,1377.95,3059.90,NaN,NaN,95.83,76498.0,42049.0,0.0,1.99,122.06,0.00,122.06,274.0,inf,0.56,16.98,AR,ATH,ELECTRICAL
58,MASFIN,326.60,399.50,-18.25,H-SC,70.83,93090.0,-4890.0,26763.0,0.37,-4.99,28.75,22.32,145.0,-0.18,0.68,35.01,XR,ATH,FINANCE
9,BAJAJFINSV,1994.80,2758.00,-1.25,X-LC,42.19,98485.0,740.0,36656.0,0.17,0.76,37.22,38.26,18.0,0.02,0.72,31.50,X40,BTT,FINANCE


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-15.28,L-SC,100.00,170014.0,28172.0,28392.0,-0.88,19.86,16.70,39.88,262.0,0.99,1.24,72.29,XY24,NTT,POWER
92,VIPIND,488.80,489.00,-212.52,H-SC,98.96,81178.0,-14138.0,14174.0,0.36,-14.83,17.46,0.04,157.0,-1.00,0.59,60.12,OX40N,NTT,MISC
3,ALKYLAMINE,2347.98,4546.37,8.61,X-SC,97.92,101063.0,100.0,94433.0,-1.99,0.10,93.44,93.63,67.0,0.00,0.74,50.53,SR,ATH,CHEMICALS
15,BLUESTARCO,1646.70,2337.55,-3.43,H-SC,96.88,181080.0,16410.0,52676.0,-0.87,9.97,29.09,41.95,124.0,0.31,1.33,18.18,X40N,ATH,AC
32,ENRIN,1377.95,3059.90,NaN,NaN,95.83,76498.0,42049.0,0.0,1.99,122.06,0.00,122.06,274.0,inf,0.56,16.98,AR,ATH,ELECTRICAL


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.98
1,25,41.96
2,50,70.59


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.33
LC    34.02
MC    21.60
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.90
X40      13.01
XY25     12.06
X40N     11.94
XR        9.99
AR        8.18
OX40N     8.18
MH        1.67
X5K       1.45
X200      1.42
SR        1.22
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.41
M-SC    15.86
H-LC    14.14
X-LC    12.96
H-MC     9.56
X-MC     7.61
M-LC     5.88
X-SC     4.44
M-MC     4.07
L-SC     2.62
L-LC     1.04
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.25,-0.48,34.14
IT,12.17,-9.38,63.10
FINANCE,8.47,-12.70,53.59
BANKS,7.71,-8.46,52.71
MISC,5.85,-20.64,77.18
PAINTS,5.69,-12.29,34.42
ELECTRICAL,5.48,1.52,44.75
HEALTHCARE,4.64,-1.70,31.41
AUTO,4.32,-14.68,65.10


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2673367.0,26
XR,1093986.0,13
AR,1031716.0,10
X40,679829.0,10
X40N,534536.0,11
XY25,471915.0,8
OX40N,310775.0,11
SR,176997.0,2
MH,87611.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1998045.0,20
M-SC,1777247.0,21
H-LC,609320.0,12
X-LC,539846.0,8
H-MC,529913.0,8
M-MC,414998.0,3
X-MC,414616.0,6
X-SC,312167.0,5
M-LC,258091.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      895386.0      6
M-SC       XY24      813762.0      7
H-SC       AR        529644.0      3
M-SC       XR        339002.0      4
M-MC       XY24      302427.0      2
H-MC       XY24      299554.0      3
X-LC       X40       273259.0      3
H-SC       XR        218443.0      3
           X40N      208657.0      4
H-LC       X40N      197334.0      4
M-SC       AR        184992.0      2
           OX40N     164808.0      5
X-MC       X40       155817.0      2
H-LC       X40       152382.0      3
L-SC       XR        152278.0      2
M-LC       XY24      150838.0      2
X-SC       XR        124434.0      1
M-SC       XY25      122129.0      1
X-LC       AR        116670.0      1
M-MC       XR        112571.0      1
X-MC       XY25      103703.0      1
X-LC       XY25      101457.0      3
X-SC       SR         94433.0      1
M-LC       XR         93254.0      1
H-SC       OX40N      88706.0      3
X-MC       XY24       86504.0      1
H-MC       XY25       82969.0      1
M-SC       SR         82564.0      1
H-LC       AR         81503.0      1
           X5K        70350.0      2
M-SC       X40        69990.0      1
H-LC       X200       65882.0      1
H-MC       AR         60746.0      1
L-SC       AR         58161.0      1
H-SC       MH         57209.0      1
L-MC       XR         54004.0      1
X-SC       XY24       52407.0      1
X-MC       X40N       52224.0      1
X-LC       X40N       48460.0      1
L-LC       XY25       47658.0      1
H-LC       XY24       41869.0      1
X-SC       OX40N      40893.0      2
H-MC       MH         30402.0      1
L-SC       XY24       28392.0      1
H-MC       X40        28381.0      1
           X40N       27861.0      1
X-MC       OX40N      16368.0      1
M-LC       XY25       13999.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 37.0 seconds
